In [1]:
%load_ext autoreload
%autoreload 2

In [3]:

import os
import sys

import torch
import numpy as np
import pandas as pd

from mpra_test.mpra_test import MPRA_Dataset

### Plant

In [22]:
name_paper = 'Plant_2024_Jores'
name_dataset = 'native'

mpra_dataset = MPRA_Dataset.load(name_paper, name_dataset)
print(mpra_dataset.shape)
print(mpra_dataset.data.columns)

(365518, 13)
Index(['X', 'Y: cold', 'Y: warm', 'Y: dark', 'Y: light', 'Y: maize',
       'obs_X: id', 'obs_X: lib', 'obs_X: species', 'obs_X: chr',
       'obs_X: loci (start)', 'obs_X: loci (end)', 'obs_X: orientation'],
      dtype='object')


In [23]:
mpra_dataset

In [24]:
print(mpra_dataset[0])

X                      ATAGAAAAATTAGGTAAAGAGTCAGTGTCGTTATGTTATGGAAGAT...
Y: cold                                                        -0.255701
Y: warm                                                         0.130293
Y: dark                                                         0.187041
Y: light                                                       -0.088923
Y: maize                                                       -0.507449
obs_X: id                                                           At-1
obs_X: lib                                                           ACR
obs_X: species                                                        At
obs_X: chr                                                             1
obs_X: loci (start)                                                 1410
obs_X: loci (end)                                                   1580
obs_X: orientation                                               Forward
Name: 0, dtype: object


In [25]:
print(mpra_dataset['obs_X: id'].shape)
print(mpra_dataset['obs_X: id'].head())

(365518,)
0      At-1
1      At-1
2     At-10
3    At-100
4    At-100
Name: obs_X: id, dtype: object


In [27]:
mpra_dataset.Y

,cold,warm,dark,light,maize
0,-0.255701,0.130293,0.187041,-0.088923,-0.507449
1,-0.189147,0.384606,0.020128,0.051520,-1.133404
2,0.039015,0.922137,0.600000,0.475028,-0.420744
3,0.450037,1.553443,2.049545,1.206474,0.345508
4,0.398944,0.386737,2.296798,1.175671,0.715303
...,...,...,...,...,...
365513,NaN,NaN,NaN,NaN,-0.576264
365514,NaN,NaN,NaN,NaN,-0.243445
365515,NaN,NaN,NaN,NaN,-0.358087
365516,NaN,NaN,NaN,NaN,-1.606015


In [31]:
print(mpra_dataset[mpra_dataset.Y['dark'] > 3])

MPRA_Dataset object with n_seq × n_readout = 13048 × 5
obs X: 'id', 'lib', 'species', 'chr', 'loci (start)', 'loci (end)', 'orientation'
obs Y: ''
readout: 'cold', 'warm', 'dark', 'light', 'maize'



In [10]:
print(mpra_dataset[:3].shape)
print(mpra_dataset[mpra_dataset['Y: dark'] > 3].shape)
print(mpra_dataset[mpra_dataset['Y: dark'] > 3]['Y: dark'].min(), mpra_dataset[mpra_dataset['Y: dark'] > 3]['Y: dark'].max())
print(mpra_dataset[mpra_dataset['obs_X: chr'].isin([2, 3, 5, 7])].shape)
print(mpra_dataset[mpra_dataset['obs_X: chr'].isin([2, 3, 5, 7])]['obs_X: chr'].value_counts())
print(mpra_dataset[np.arange(3)].shape)
print(mpra_dataset[torch.arange(3)].shape)
print(mpra_dataset[[0,1,2]].shape)
print(mpra_dataset['Y: dark'].shape)
print(mpra_dataset[['Y: cold', 'Y: warm', 'Y: dark', 'Y: light']].shape)

(3, 13)
(13048, 13)
3.000056731135264 7.282275582325817
(172470, 13)
obs_X: chr
3    52966
5    50589
2    49517
7    19398
Name: count, dtype: int64
(3, 13)
(3, 13)
(3, 13)
(365518,)
(365518, 4)


In [11]:
print(mpra_dataset[:3].shape)
print(mpra_dataset[mpra_dataset.Y['dark'] > 3].shape)
print(mpra_dataset[mpra_dataset.Y['dark'] > 3].Y['dark'].min(), mpra_dataset[mpra_dataset.Y['dark'] > 3].Y['dark'].max())
print(mpra_dataset[mpra_dataset.obs_X['chr'].isin([2, 3, 5, 7])].shape)
print(mpra_dataset[mpra_dataset.obs_X['chr'].isin([2, 3, 5, 7])].obs_X['chr'].value_counts())
print(mpra_dataset[np.arange(3)].shape)
print(mpra_dataset[torch.arange(3)].shape)
print(mpra_dataset[[0,1,2]].shape)
print(mpra_dataset.Y['dark'].shape)
print(mpra_dataset.Y[['cold', 'warm', 'dark', 'light']].shape)

(3, 13)
(13048, 13)
3.000056731135264 7.282275582325817
(172470, 13)
chr
3    52966
5    50589
2    49517
7    19398
Name: count, dtype: int64
(3, 13)
(3, 13)
(3, 13)
(365518,)
(365518, 4)


In [12]:
chr_valid = [4, 9]
chr_infer = [2, 7]

idx_valid = mpra_dataset['obs_X: chr'].isin(chr_valid)
idx_infer = mpra_dataset['obs_X: chr'].isin(chr_infer)
idx_train = ~ (idx_valid | idx_infer)


In [13]:
batch_size = 64
num_workers = 4
dataloader_train = mpra_dataset[idx_train].to_DataLoader(
    cols_Y = ['Y: cold', 'Y: warm', 'Y: dark', 'Y: light'],
    batch_size = batch_size, num_workers = num_workers, shuffle = True, 
)
dataloader_valid = mpra_dataset[idx_valid].to_DataLoader(
    cols_Y = ['Y: cold', 'Y: warm', 'Y: dark', 'Y: light'],
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)
dataloader_infer = mpra_dataset[idx_infer].to_DataLoader(
    cols_Y = ['Y: cold', 'Y: warm', 'Y: dark', 'Y: light'],
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)

print('len(dataloader_train.dataset): ', len(dataloader_train.dataset))
print('len(dataloader_valid.dataset): ', len(dataloader_valid.dataset))
print('len(dataloader_infer.dataset): ', len(dataloader_infer.dataset))

len(dataloader_train.dataset):  216247
len(dataloader_valid.dataset):  57836
len(dataloader_infer.dataset):  63737


### Nature 2022 Regev

In [14]:
name_paper = 'Nature_2022_Regev'
name_dataset = 'test_complex'

mpra_dataset = MPRA_Dataset.load(name_paper, name_dataset)
print(mpra_dataset.shape)
print(mpra_dataset.data.columns)

(3331, 2)
Index(['X', 'Y: expr'], dtype='object')


In [15]:
n = len(mpra_dataset)
idx_valid, idx_infer, idx_train = np.split(np.random.permutation(n), [int(0.25*n), int(0.50*n)])

In [16]:
batch_size = 64
num_workers = 4
dataloader_train = mpra_dataset[idx_train].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = True, 
)
dataloader_valid = mpra_dataset[idx_valid].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)
dataloader_infer = mpra_dataset[idx_infer].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)

print('len(dataloader_train.dataset): ', len(dataloader_train.dataset))
print('len(dataloader_valid.dataset): ', len(dataloader_valid.dataset))
print('len(dataloader_infer.dataset): ', len(dataloader_infer.dataset))

ValueError: all input arrays must have the same shape

### Genome Research 2017 Seelig

In [17]:
name_paper = 'GenomeResearch_2017_Seelig'
name_dataset = 'random'

mpra_dataset = MPRA_Dataset.load(name_paper, name_dataset)
print(mpra_dataset.shape)
print(mpra_dataset.data.columns)

(489348, 4)
Index(['X', 'Y: expr', 'obs_Y: raw count (input)',
       'obs_Y: raw count (output)'],
      dtype='object')


In [18]:
n = len(mpra_dataset)
idx_valid, idx_infer, idx_train = np.split(np.random.permutation(n), [int(0.25*n), int(0.50*n)])

In [19]:
batch_size = 64
num_workers = 4
dataloader_train = mpra_dataset[idx_train].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = True, 
)
dataloader_valid = mpra_dataset[idx_valid].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)
dataloader_infer = mpra_dataset[idx_infer].to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)

print('len(dataloader_train.dataset): ', len(dataloader_train.dataset))
print('len(dataloader_valid.dataset): ', len(dataloader_valid.dataset))
print('len(dataloader_infer.dataset): ', len(dataloader_infer.dataset))

len(dataloader_train.dataset):  244674
len(dataloader_valid.dataset):  122337
len(dataloader_infer.dataset):  122337


In [20]:
name_dataset_pred = 'native'
mpra_dataset_pred = MPRA_Dataset.load(name_paper, name_dataset_pred)

batch_size = 64
num_workers = 4
dataloader_pred = mpra_dataset_pred.to_DataLoader(
    batch_size = batch_size, num_workers = num_workers, shuffle = False, 
)

ValueError: all input arrays must have the same shape